In [613]:
# 노드 4-1. 미니 프로젝트 - 영국시장의 중고 자동차 가격 데이터 다루기
# 주 데이터는 cars.csv, 보조 데이터 brand.csv
# 중고차 가격 예측 모델, 중고차 특성에 따른 클러스터링 등 가정하여 데이터 준비. 

In [614]:
#### 완료 1. cars.csv와 brand.csv의 데이터 합치기
#### 완료 2. 카테고리형 변수를 숫자 형태로 변환하기
#### 완료 3. 결측치 처리하기
#### 완료 4. Scaler 활용해보기
#### 완료 5. 기존 데이터 정보의 70% 이상을 가지는 수준에서 최소한의 주성분 추출(PCA)
# 여유가 되면 실습할 내용
#### 완료 1. 국가별 총 브랜드 개수는?
#### 완료 2. 상관관계가 높은 변수들이 있는지 확인


In [615]:
# sklearn.preprocessing의 LabelEncoder는 카테고리형 변수를 숫자로 변환시 사용. 
# 머신러닝 모델이 숫자 데이터를 다룸.
# StandardScaler는 데이터를 정규화.feature를 평균 0, 분산 1이 되도록 변환. 
# sklearn.preprocessing의 PCA는 데이터의 차원을 축소. 

import pandas as pd
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA

In [616]:
# 1. cars.csv와 brand.csv의 데이터 합치기
#  cars.csv의 title 변수로부터 브랜드 데이터를 추출

cars = pd.read_csv('~/data/cars.csv')
brand = pd.read_csv('~/data/brand.csv')

In [617]:
cars.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3687 entries, 0 to 3686
Data columns (total 13 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   title              3687 non-null   object 
 1   Price              3687 non-null   int64  
 2   Mileage(miles)     3687 non-null   int64  
 3   Registration_Year  3687 non-null   int64  
 4   Previous Owners    2276 non-null   float64
 5   Fuel type          3687 non-null   object 
 6   Body type          3687 non-null   object 
 7   Engine             3642 non-null   object 
 8   Gearbox            3687 non-null   object 
 9   Doors              3662 non-null   float64
 10  Seats              3652 non-null   float64
 11  Emission Class     3600 non-null   object 
 12  Service history    540 non-null    object 
dtypes: float64(3), int64(3), object(7)
memory usage: 374.6+ KB


In [618]:
cars.head()

,title,Price,Mileage(miles),Registration_Year,Previous Owners,Fuel type,Body type,Engine,Gearbox,Doors,Seats,Emission Class,Service history
0,SKODA FABIA,6900,70189,2016,3.0,Diesel,Hatchback,1.4L,Manual,5.0,5.0,Euro 6,NaN
1,VAUXHALL CORSA,1495,88585,2008,4.0,Petrol,Hatchback,1.2L,Manual,3.0,5.0,Euro 4,Full
2,HYUNDAI I30,949,137000,2011,NaN,Petrol,Hatchback,1.4L,Manual,5.0,5.0,Euro 5,NaN
3,MINI HATCH,2395,96731,2010,5.0,Petrol,Hatchback,1.4L,Manual,3.0,4.0,Euro 4,Full
4,VAUXHALL CORSA,1000,85000,2013,NaN,Diesel,Hatchback,1.3L,Manual,5.0,5.0,Euro 5,NaN


In [619]:
btc = cars['Body type'].value_counts()
btc

Hatchback      2280
SUV             461
Saloon          368
Estate          171
MPV             153
Coupe           139
Convertible     109
Pickup            3
Combi Van         2
Minibus           1
Name: Body type, dtype: int64

In [620]:
bc = cars['title'].value_counts()
bc

VAUXHALL CORSA                                   223
FORD FOCUS                                       162
VAUXHALL ASTRA                                   155
FORD FIESTA                                      128
BMW 3 SERIES                                     119
                                                ... 
PROTON PROTON                                      1
ABARTH FIAT 595 1.4 TJET (165PS) SCORPIONEORO      1
KIA STONIC                                         1
TOYOTA PRIUS+                                      1
HONDA PRELUDE                                      1
Name: title, Length: 455, dtype: int64

In [621]:
brand.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 48 entries, 0 to 47
Data columns (total 2 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   title    48 non-null     object
 1   country  48 non-null     object
dtypes: object(2)
memory usage: 896.0+ bytes


In [622]:
brand.head()

,title,country
0,skoda,Czech Republic
1,vauxhall,United Kingdom
2,hyundai,South Korea
3,mini,United Kingdom
4,ford,United States


In [623]:
cars.head()

,title,Price,Mileage(miles),Registration_Year,Previous Owners,Fuel type,Body type,Engine,Gearbox,Doors,Seats,Emission Class,Service history
0,SKODA FABIA,6900,70189,2016,3.0,Diesel,Hatchback,1.4L,Manual,5.0,5.0,Euro 6,NaN
1,VAUXHALL CORSA,1495,88585,2008,4.0,Petrol,Hatchback,1.2L,Manual,3.0,5.0,Euro 4,Full
2,HYUNDAI I30,949,137000,2011,NaN,Petrol,Hatchback,1.4L,Manual,5.0,5.0,Euro 5,NaN
3,MINI HATCH,2395,96731,2010,5.0,Petrol,Hatchback,1.4L,Manual,3.0,4.0,Euro 4,Full
4,VAUXHALL CORSA,1000,85000,2013,NaN,Diesel,Hatchback,1.3L,Manual,5.0,5.0,Euro 5,NaN


In [624]:
# cars 테이블의 title 컬럼을 brand와 name으로 분리
cars['brand'] = cars['title'].apply(lambda x: x.split(' ')[0])
cars['name'] = cars['title'].apply(lambda x: x.split(' ')[1])
cars.head()

,title,Price,Mileage(miles),Registration_Year,Previous Owners,Fuel type,Body type,Engine,Gearbox,Doors,Seats,Emission Class,Service history,brand,name
0,SKODA FABIA,6900,70189,2016,3.0,Diesel,Hatchback,1.4L,Manual,5.0,5.0,Euro 6,NaN,SKODA,FABIA
1,VAUXHALL CORSA,1495,88585,2008,4.0,Petrol,Hatchback,1.2L,Manual,3.0,5.0,Euro 4,Full,VAUXHALL,CORSA
2,HYUNDAI I30,949,137000,2011,NaN,Petrol,Hatchback,1.4L,Manual,5.0,5.0,Euro 5,NaN,HYUNDAI,I30
3,MINI HATCH,2395,96731,2010,5.0,Petrol,Hatchback,1.4L,Manual,3.0,4.0,Euro 4,Full,MINI,HATCH
4,VAUXHALL CORSA,1000,85000,2013,NaN,Diesel,Hatchback,1.3L,Manual,5.0,5.0,Euro 5,NaN,VAUXHALL,CORSA


In [625]:
# brand 테이블의 title 컬럼을 cars 테이블과 합치기 위해 같은 대문자로 변경하고 컬럼명을 cars 테이블과 같은 brand로 변경.
brand['title'] = brand['title'].str.upper()
brand.rename(columns = {'title': 'brand'}, inplace = True)
brand.head()

,brand,country
0,SKODA,Czech Republic
1,VAUXHALL,United Kingdom
2,HYUNDAI,South Korea
3,MINI,United Kingdom
4,FORD,United States


In [626]:
# cars와 brand의 두 테이블에서 brand 컬럼으로 데이터를 합치기.
merged_data = pd.merge(cars, brand, on = 'brand', how = 'left')
merged_data.head()

,title,Price,Mileage(miles),Registration_Year,Previous Owners,Fuel type,Body type,Engine,Gearbox,Doors,Seats,Emission Class,Service history,brand,name,country
0,SKODA FABIA,6900,70189,2016,3.0,Diesel,Hatchback,1.4L,Manual,5.0,5.0,Euro 6,NaN,SKODA,FABIA,Czech Republic
1,VAUXHALL CORSA,1495,88585,2008,4.0,Petrol,Hatchback,1.2L,Manual,3.0,5.0,Euro 4,Full,VAUXHALL,CORSA,United Kingdom
2,HYUNDAI I30,949,137000,2011,NaN,Petrol,Hatchback,1.4L,Manual,5.0,5.0,Euro 5,NaN,HYUNDAI,I30,South Korea
3,MINI HATCH,2395,96731,2010,5.0,Petrol,Hatchback,1.4L,Manual,3.0,4.0,Euro 4,Full,MINI,HATCH,United Kingdom
4,VAUXHALL CORSA,1000,85000,2013,NaN,Diesel,Hatchback,1.3L,Manual,5.0,5.0,Euro 5,NaN,VAUXHALL,CORSA,United Kingdom


In [627]:
cars.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3687 entries, 0 to 3686
Data columns (total 15 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   title              3687 non-null   object 
 1   Price              3687 non-null   int64  
 2   Mileage(miles)     3687 non-null   int64  
 3   Registration_Year  3687 non-null   int64  
 4   Previous Owners    2276 non-null   float64
 5   Fuel type          3687 non-null   object 
 6   Body type          3687 non-null   object 
 7   Engine             3642 non-null   object 
 8   Gearbox            3687 non-null   object 
 9   Doors              3662 non-null   float64
 10  Seats              3652 non-null   float64
 11  Emission Class     3600 non-null   object 
 12  Service history    540 non-null    object 
 13  brand              3687 non-null   object 
 14  name               3687 non-null   object 
dtypes: float64(3), int64(3), object(9)
memory usage: 432.2+ KB


In [628]:
# 2. 카테고리형 변수를 숫자 형태로 변환하기
#  Engine, Emission Class 변수에 주목
pd.get_dummies(merged_data, columns = ['Engine', 'Emission Class'], drop_first = True)

,title,Price,Mileage(miles),Registration_Year,Previous Owners,Fuel type,Body type,Gearbox,Doors,Seats,...,Engine_4.4L,Engine_4.8L,Engine_5.0L,Engine_5.5L,Engine_6.3L,Emission Class_Euro 2,Emission Class_Euro 3,Emission Class_Euro 4,Emission Class_Euro 5,Emission Class_Euro 6
0,SKODA FABIA,6900,70189,2016,3.0,Diesel,Hatchback,Manual,5.0,5.0,...,0,0,0,0,0,0,0,0,0,1
1,VAUXHALL CORSA,1495,88585,2008,4.0,Petrol,Hatchback,Manual,3.0,5.0,...,0,0,0,0,0,0,0,1,0,0
2,HYUNDAI I30,949,137000,2011,NaN,Petrol,Hatchback,Manual,5.0,5.0,...,0,0,0,0,0,0,0,0,1,0
3,MINI HATCH,2395,96731,2010,5.0,Petrol,Hatchback,Manual,3.0,4.0,...,0,0,0,0,0,0,0,1,0,0
4,VAUXHALL CORSA,1000,85000,2013,NaN,Diesel,Hatchback,Manual,5.0,5.0,...,0,0,0,0,0,0,0,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3682,RENAULT MEGANE,1395,76202,2006,4.0,Petrol,Hatchback,Manual,5.0,5.0,...,0,0,0,0,0,0,0,1,0,0
3683,AUDI A4,6990,119000,2012,NaN,Petrol,Saloon,Manual,4.0,5.0,...,0,0,0,0,0,0,0,0,1,0
3684,BMW 3 SERIES,3995,139000,2013,NaN,Diesel,Saloon,Manual,4.0,5.0,...,0,0,0,0,0,0,0,0,1,0
3685,HONDA ACCORD,1390,179190,2007,NaN,Diesel,Estate,Manual,5.0,5.0,...,0,0,0,0,0,0,0,1,0,0


In [629]:
# 3. 결측치 처리하기
#  Service history 변수의 결측치는 'Unknown'으로 채우고, 
#  결측치 변수가 일정 개수 이상 포함된 데이터는 제거

In [630]:
merged_data['Service history'].isnull().count()

3687

In [631]:
merged_data['Service history'].count()

540

In [632]:
merged_data['Service history'].fillna('Unknown', inplace=True)

In [633]:
merged_data['Service history'].unique()

array(['Unknown', 'Full'], dtype=object)

In [634]:
merged_data['Service history'].isnull().sum()

0

In [635]:
merged_data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 3687 entries, 0 to 3686
Data columns (total 16 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   title              3687 non-null   object 
 1   Price              3687 non-null   int64  
 2   Mileage(miles)     3687 non-null   int64  
 3   Registration_Year  3687 non-null   int64  
 4   Previous Owners    2276 non-null   float64
 5   Fuel type          3687 non-null   object 
 6   Body type          3687 non-null   object 
 7   Engine             3642 non-null   object 
 8   Gearbox            3687 non-null   object 
 9   Doors              3662 non-null   float64
 10  Seats              3652 non-null   float64
 11  Emission Class     3600 non-null   object 
 12  Service history    3687 non-null   object 
 13  brand              3687 non-null   object 
 14  name               3687 non-null   object 
 15  country            3687 non-null   object 
dtypes: float64(3), int64(3),

In [636]:
# 16개 컬럼 중에 결측치 변수가 4개일 때 1개 행 삭제 
merged_data.dropna(thresh=12, inplace=True)
merged_data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 3686 entries, 0 to 3686
Data columns (total 16 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   title              3686 non-null   object 
 1   Price              3686 non-null   int64  
 2   Mileage(miles)     3686 non-null   int64  
 3   Registration_Year  3686 non-null   int64  
 4   Previous Owners    2276 non-null   float64
 5   Fuel type          3686 non-null   object 
 6   Body type          3686 non-null   object 
 7   Engine             3642 non-null   object 
 8   Gearbox            3686 non-null   object 
 9   Doors              3662 non-null   float64
 10  Seats              3652 non-null   float64
 11  Emission Class     3600 non-null   object 
 12  Service history    3686 non-null   object 
 13  brand              3686 non-null   object 
 14  name               3686 non-null   object 
 15  country            3686 non-null   object 
dtypes: float64(3), int64(3),

In [637]:
# 16개 컬럼 중에 결측치 변수가 3개일 때 13개 행 삭제 
merged_data.dropna(thresh=13, inplace=True)
merged_data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 3674 entries, 0 to 3686
Data columns (total 16 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   title              3674 non-null   object 
 1   Price              3674 non-null   int64  
 2   Mileage(miles)     3674 non-null   int64  
 3   Registration_Year  3674 non-null   int64  
 4   Previous Owners    2267 non-null   float64
 5   Fuel type          3674 non-null   object 
 6   Body type          3674 non-null   object 
 7   Engine             3639 non-null   object 
 8   Gearbox            3674 non-null   object 
 9   Doors              3662 non-null   float64
 10  Seats              3652 non-null   float64
 11  Emission Class     3600 non-null   object 
 12  Service history    3674 non-null   object 
 13  brand              3674 non-null   object 
 14  name               3674 non-null   object 
 15  country            3674 non-null   object 
dtypes: float64(3), int64(3),

In [638]:
# Previous Owners, Doors, Seats 세 개의 컬럼의 결측치 삭제 결정.
merged_data = merged_data.dropna()
merged_data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2239 entries, 0 to 3686
Data columns (total 16 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   title              2239 non-null   object 
 1   Price              2239 non-null   int64  
 2   Mileage(miles)     2239 non-null   int64  
 3   Registration_Year  2239 non-null   int64  
 4   Previous Owners    2239 non-null   float64
 5   Fuel type          2239 non-null   object 
 6   Body type          2239 non-null   object 
 7   Engine             2239 non-null   object 
 8   Gearbox            2239 non-null   object 
 9   Doors              2239 non-null   float64
 10  Seats              2239 non-null   float64
 11  Emission Class     2239 non-null   object 
 12  Service history    2239 non-null   object 
 13  brand              2239 non-null   object 
 14  name               2239 non-null   object 
 15  country            2239 non-null   object 
dtypes: float64(3), int64(3),

In [639]:
# 4. Scaler 활용해보기

In [640]:
merged_data.describe()

,Price,Mileage(miles),Registration_Year,Previous Owners,Doors,Seats
count,2239.000000,2.239000e+03,2239.000000,2239.000000,2239.000000,2239.000000
mean,5984.423403,8.293097e+04,2012.308620,2.805270,4.410004,4.930773
std,4761.228187,4.240144e+04,4.710367,1.540791,0.937172,0.591974
min,400.000000,6.000000e+00,1993.000000,1.000000,2.000000,2.000000
25%,2450.000000,5.839800e+04,2009.000000,2.000000,4.000000,5.000000
50%,4250.000000,8.000000e+04,2012.000000,3.000000,5.000000,5.000000
75%,8492.000000,1.050000e+05,2016.000000,4.000000,5.000000,5.000000
max,33900.000000,1.110100e+06,2023.000000,9.000000,5.000000,7.000000


In [641]:
merged_data.isna().sum()

title                0
Price                0
Mileage(miles)       0
Registration_Year    0
Previous Owners      0
Fuel type            0
Body type            0
Engine               0
Gearbox              0
Doors                0
Seats                0
Emission Class       0
Service history      0
brand                0
name                 0
country              0
dtype: int64

In [642]:
merged_data['Previous Owners'].unique()

array([3., 4., 5., 1., 2., 6., 7., 9., 8.])

In [643]:
ss = StandardScaler()
scaled_data = ss.fit_transform(merged_data.select_dtypes(include=['float64','int64']))

In [648]:
# 5. 기존 데이터 정보의 70% 이상을 가지는 수준에서 최소한의 주성분 추출(PCA)
pca = PCA(n_components=0.70)
pca.fit(scaled_data)
pca.transform(scaled_data)
pca_data

array([[-0.72254965, -0.4183744 , -0.30775859],
       [ 1.67294883,  0.77142047, -0.40482917],
       [ 1.78734085,  1.94903797, -0.42902522],
       ...,
       [ 1.02300518, -0.71210684,  0.19789677],
       [ 1.42464245, -0.60875244, -0.74760798],
       [ 1.45218488, -0.63066355, -1.84117654]])

In [651]:
pd.DataFrame(pca.transform(scaled_data))

,0,1,2
0,-0.722550,-0.418374,-0.307759
1,1.672949,0.771420,-0.404829
2,1.787341,1.949038,-0.429025
3,2.016230,1.796581,0.866525
4,1.592974,-0.663756,-0.962918
...,...,...,...
2234,1.450725,-0.679478,0.068246
2235,3.093663,2.396356,1.791248
2236,1.023005,-0.712107,0.197897
2237,1.424642,-0.608752,-0.747608


In [650]:
# 여유가 되면 실습할 내용
#### 완료 1. 국가별 총 브랜드 개수는?
#### 완료 2. 상관관계가 높은 변수들이 있는지 확인

In [667]:
merged_data['brand'].nunique()

44

In [658]:
# 1. 국가별 총 브랜드 개수는?
merged_data.groupby('country')['brand'].nunique()

country
Czech Republic     1
France             4
Germany            6
Italy              4
Japan             10
Malaysia           1
Romania            1
South Korea        4
Spain              1
Sweden             2
United Kingdom     6
United States      4
Name: brand, dtype: int64

In [661]:
# 2. 상관관계가 높은 변수들이 있는지 확인
merged_data.corr()

,Price,Mileage(miles),Registration_Year,Previous Owners,Doors,Seats
Price,1.000000,-0.490992,0.786851,-0.486335,0.101333,-0.021583
Mileage(miles),-0.490992,1.000000,-0.460418,0.317762,-0.034027,0.070788
Registration_Year,0.786851,-0.460418,1.000000,-0.516012,0.218907,0.047566
Previous Owners,-0.486335,0.317762,-0.516012,1.000000,-0.148605,-0.023114
Doors,0.101333,-0.034027,0.218907,-0.148605,1.000000,0.489328
Seats,-0.021583,0.070788,0.047566,-0.023114,0.489328,1.000000


In [666]:
# Price와 Registration_Year이 0.786851의 높은 상관관계가 있습니다.